### 训练集和测试集特征分离后   baseline : 0.713741

1、单独加入广告发布的不同用户数，0.716053，效果有较大提升

2、加入广告的出现次数，0.713612,效果下降

2、加入广告点击率和广告发布的不同用户数， 0.714031, 效果下降。。

2、加入campaign的点击率，0.715376，效果下降

3、加入用户所在LBS的历史点击率, 效果下降。。

4、加入用户的兴趣总数，0.714661，效果下降。。

5、加入用户的历史接收的广告次数与历史点击次数，以及历史点击率, 效果均下降

6、分析广告投放和地理位置的关系

7、加入每个地理位置的人数特征, 效果下降

8、考虑点击相同广告的用户的interest2分布,在点击的用户中，某个兴趣id出现次数是总人数的三分之一则保留

In [3]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt

data = pd.read_csv('./datasets/train_data.csv')
target = pd.read_csv('./datasets/train_target.csv', names=['label'], header=None)

data.drop(data.columns[[0]], axis=1, inplace=True)  #删除某列函数
target = target.reset_index(drop=True)

one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility', 'education','gender','house','os','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']



for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, np.array(target).squeeze(), random_state=42, test_size=0.3)

In [5]:
data_clicked = X_train[y_train == 1]

In [6]:
# 处理联网类型特征
ct_train = X_train['ct'].values
ct_train = [m.split(' ') for m in ct_train]
ct_trains = []
for i in ct_train:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_trains.append(index)

ct_test = X_test['ct'].values
ct_test = [m.split(' ') for m in ct_test]
ct_tests = []
for i in ct_test:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_tests.append(index)


增加每个广告点击率特征

In [7]:
num_ad = X_train['aid'].value_counts().sort_index()
num_ad_clicked = data_clicked['aid'].value_counts().sort_index()

ratio = num_ad_clicked / num_ad

ratio_clicked = pd.DataFrame({
    'aid': ratio.index,
    'ratio_clicked' : ratio.values
})


In [8]:
X_train = pd.merge(X_train, ratio_clicked, on=['aid'], how='left')
X_test = pd.merge(X_test, ratio_clicked, on=['aid'], how='left')

增加每个广告出现次数

In [9]:
num_ad_post = X_train['aid'].value_counts().sort_index()

num_ad_post = pd.DataFrame({
    'aid': num_ad_post.index,
    'num_ad_post' : num_ad_post.values
})

In [10]:
X_train = pd.merge(X_train, num_ad_post, on=['aid'], how='left')
X_test = pd.merge(X_test, num_ad_post, on=['aid'], how='left')

增加每个广告推送给不同的用户数

In [11]:
num_advertise_touser = X_train.groupby('aid').uid.nunique()
num_advertise_touser = pd.DataFrame({
    'aid': num_advertise_touser.index,
    'num_advertise_touser' : num_advertise_touser.values
})
X_train = pd.merge(X_train, num_advertise_touser, on=['aid'], how='left')
X_test = pd.merge(X_test, num_advertise_touser, on=['aid'], how='left')

加入推广计划转化率

In [12]:
num_campaign = X_train['campaignId'].value_counts().sort_index()
num_campaign_clicked = data_clicked['campaignId'].value_counts().sort_index()

ratio_num_campaign = num_campaign_clicked / num_campaign

ratio_num_campaign = pd.DataFrame({
    'campaignId': ratio_num_campaign.index,
    'ratio_num_campaign' : ratio_num_campaign.values
})
X_train = pd.merge(X_train, ratio_num_campaign, on=['campaignId'], how='left')
X_test = pd.merge(X_test, ratio_num_campaign, on=['campaignId'], how='left')

加入用户兴趣总数

In [56]:
train_all_interest = X_train[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
test_all_interest = X_test[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
num_all_interest_train = []
num_all_interest_test = []
aids = []
train_array = np.array(train_all_interest)
test_array = np.array(test_all_interest)

for i in range(train_array.shape[0]):
    num = 0
    inter = train_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_all_interest_train.append(num)
    
    
for i in range(test_array.shape[0]):
    num = 0
    inter = test_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_all_interest_test.append(num)

In [59]:
num_all_interest_train = pd.DataFrame(num_all_interest_train, columns=['num_all_interest'])
num_all_interest_test = pd.DataFrame(num_all_interest_test, columns=['num_all_interest'])

X_train = pd.concat([X_train, num_all_interest_train], axis=1)
X_test = pd.concat([X_test, num_all_interest_test], axis=1)

加入地理位置所对应的用户人数

In [ ]:
num_lbs_user = X_train.groupby('LBS').uid.nunique()
num_lbs_user = pd.DataFrame({
    'LBS': num_lbs_user.index,
    'num_lbs_user' : num_lbs_user.values
})
X_train = pd.merge(X_train, num_lbs_user, on=['LBS'], how='left')
X_test = pd.merge(X_test, num_lbs_user, on=['LBS'], how='left')

加入用户所在LBS的历史点击率

In [14]:
num_lbs = X_train.groupby('LBS').uid.nunique()
num_lbs_clicked = data_clicked.groupby('LBS').uid.nunique()
ratio_num_lbs = num_lbs_clicked / num_lbs
ratio_num_lbs = ratio_num_lbs.fillna(0)
ratio_num_lbs = pd.DataFrame({
    'LBS': ratio_num_lbs.index,
    'ration_num_LBS' : ratio_num_lbs.values
})
X_train = pd.merge(X_train, ratio_num_lbs, on=['LBS'], how='left')
X_test = pd.merge(X_test, ratio_num_lbs, on=['LBS'], how='left')

加入用户的历史接收的广告次数与历史点击次数，以及历史点击率

In [15]:
num_user_getad = X_train.groupby('uid').aid.nunique().sort_index()
num_user_clicked = data_clicked.groupby('uid').aid.nunique().sort_index()

num_user_getad = pd.DataFrame({
    'uid': num_user_getad.index,
    'num_user_getad' : num_user_getad.values
})

num_user_clicked = pd.DataFrame({
    'uid': num_user_clicked.index,
    'num_user_clicked' : num_user_clicked.values
})

In [16]:
X_train = pd.merge(X_train, num_user_getad, on=['uid'], how='left')
X_test = pd.merge(X_test, num_user_getad, on=['uid'], how='left')

In [182]:
X_train = pd.merge(X_train, num_user_clicked, on=['uid'], how='left')
X_test = pd.merge(X_test, num_user_clicked, on=['uid'], how='left')

In [17]:
X_train.fillna(0, inplace=True)

In [18]:
X_test.fillna(0, inplace=True)

In [151]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train[['num_advertise_touser'
                        ]].values)
X_train_encoded = scaler.transform(X_train[['num_advertise_touser'
                                                 ]].values)

X_test_encoded = scaler.transform(X_test[['num_advertise_touser'
                                                  ]].values)

X_train_encoded= np.hstack((X_train_encoded, ct_trains))
X_test_encoded= np.hstack((X_test_encoded, ct_tests))


# X_train_encoded = ct_trains
# X_test_encoded = ct_tests

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [152]:
oc_encoder = OneHotEncoder()
for feature in one_hot_feature:
    oc_encoder.fit(X_train[feature].reshape(-1, 1))
    train_a=oc_encoder.transform(X_train[feature].values.reshape(-1, 1))
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a=oc_encoder.transform(X_test[feature].values.reshape(-1, 1))
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [158]:
X_train_encoded.toarray().shape

(615917, 1177)

In [ ]:
vector_feature=['interest1','interest2_new', 'interest5','kw1','kw2','topic1','topic2']
ct_encoder = CountVectorizer(min_df=0.0009)
for feature in vector_feature:
    ct_encoder.fit(X_train[feature])
    train_a = ct_encoder.transform(X_train[feature])
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a = ct_encoder.transform(X_test[feature])
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

In [ ]:
# y_train = np.array(target).squeeze()
clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=0.9,
        max_depth=-1, n_estimators=500, objective='binary',
        subsample=0.7, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.05, min_child_weight=60, random_state=2018, n_jobs=-1
    )
clf.fit(X_train_encoded, y_train, eval_set=[(X_test_encoded, y_test)], eval_metric='auc',early_stopping_rounds=50)

In [73]:
num_user_clicksameAd_interest = data_clicked.groupby('aid').interest2.value_counts()

In [74]:
num_adid = data_clicked['aid'].value_counts().sort_index().index

In [75]:
dict_interest2 = {}
for adid in num_adid:
    dict_buf = {}
    for interest in num_user_clicksameAd_interest.items():
        index = interest[0]
        if index[0] == adid:
            number = interest[1]
            detail = index[1]
            detail = detail.split(' ')
            for det in detail:
                if det not in dict_buf:
                    dict_buf[det] = number
                else:
                    dict_buf[det] += number
    dict_interest2[adid] = dict_buf

In [76]:
num_aid_clicked = dict(data_clicked['aid'].value_counts().sort_index())

In [77]:
dict_common_interest2 = []
for adid, dict_inter in dict_interest2.items():
    dict_common_buf = {}
    dict_common_buf['aid'] = adid
    common_inter = []
    ad_total = num_ad_clicked[adid] - dict_inter.get('-1', 0)
    if '-1' in dict_inter:
        dict_inter.pop('-1')
    for id_inter, num in dict_inter.items():
            if num >= ad_total/4:
                common_inter.append(id_inter)
    dict_common_buf['common_interest2'] = common_inter
    dict_common_interest2.append(dict_common_buf)

In [78]:
df_common_interest2 = pd.DataFrame(dict_common_interest2)

In [79]:
X_train = pd.merge(X_train, df_common_interest2, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest2, on=['aid'], how='left')

In [91]:
train_user_interest2 = X_train[['interest2', 'common_interest2']]
test_user_interest2 = X_test[['interest2', 'common_interest2']]

In [137]:
train_interest2_new = []
for i in train_user_interest2.values:
    inter2_buf = []
    int2 = i[0].split(' ')
    common = i[1]
    for det in int2:
        if det in common:
            inter2_buf.append(det)
    inter2_buf.extend(int2)
    train_interest2_new.append(inter2_buf)

In [155]:
test_interest2_new = []
for i in test_user_interest2.values:
    inter2_buf = []
    int2 = i[0].split(' ')
    common = i[1]
    for det in int2:
        if det in common:
            inter2_buf.append(det)
    inter2_buf.extend(int2)
    test_interest2_new.append(inter2_buf)

In [170]:
train_interest2_new = [' '.join(m) for m in train_interest2_new]
X_train['interest2_new'] = train_interest2_new

In [148]:
X_train.drop('interest_new', axis=1, inplace=True)

In [167]:
test_interest2_new = [' '.join(m) for m in test_interest2_new]
X_test['interest2_new'] = test_interest2_new